# Exploratory Data Analysis
### 1. Benign URL Data 
- Data Type
- Data Shape

### 2. Phishin URL Data
- Data Type
- Data Shape
    
# Feature Engneering
- Digit Count of URL
- URL 길이
- Checking URL is Typosquatted or not (google.com --> goggle.com)
- Subdomain 개수
- Top level Domain 이 자주 쓰이는 Domain인가?
- 등록후 지난 일수
- http:// or https://
- #of http:// or https://
- 특수문자(#, =, /,&)
- com 개수

# Feature Importance
- XGBClassifier Model 

-----------------------------------------------------
### 1. Benign Data
- Data Type
- Data Shape

In [35]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse, parse_qs

In [5]:
df_benign = pd.read_csv("../test/url_0.csv")
df_benign.tail()

,1,http://www.liquidgeneration.com/,Adult
1048570,1048572,https://www.nucleics.com/peaktrace/,Science
1048571,1048573,http://www.phenosystems.com,Science
1048572,1048574,http://www.thebiolog.com,Science
1048573,1048575,http://code.google.com/p/openbiomind/,Science
1048574,1048576,http://va-genex.sourceforge.net/,Science


In [7]:
list(df_benign['Adult'].unique())

['Adult',
 'Arts',
 'Business',
 'Computers',
 'Games',
 'Health',
 'Home',
 'Kids',
 'News',
 'Recreation',
 'Reference',
 'Science']

In [8]:
df_benign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
1                                   1048575 non-null int64
http://www.liquidgeneration.com/    1048573 non-null object
Adult                               1048575 non-null object
dtypes: int64(1), object(2)
memory usage: 24.0+ MB


In [11]:
df_benign.shape

(1048575, 3)

In [14]:
# from sklearn.utils import shuffle
# df = shuffle(df)
# df_0 = df[:15000].reset_index(drop=True)
# df_0

In [13]:
df_pishing = pd.read_csv("../test/url_1.csv")
df_pishing.tail()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
15256,1095720,https://sites.google.com/site/freehabbocoinsgb...,http://www.phishtank.com/phish_detail.php?phis...,2010-12-30T12:13:46+00:00,yes,2010-12-30T15:29:19+00:00,yes,Sulake Corporation
15257,876840,http://mundovirtualhabbo.blogspot.com/2009_01_...,http://www.phishtank.com/phish_detail.php?phis...,2009-12-03T10:40:26+00:00,yes,2009-12-03T23:56:01+00:00,yes,Sulake Corporation
15258,549159,http://aijcs.blogspot.com/2005/03/colourful-li...,http://www.phishtank.com/phish_detail.php?phis...,2008-11-07T15:04:00+00:00,yes,2011-09-03T19:15:33+00:00,yes,Other
15259,530149,http://tnet.at.ua/index/0-13,http://www.phishtank.com/phish_detail.php?phis...,2008-10-20T09:43:48+00:00,yes,2008-10-21T00:33:18+00:00,yes,Other
15260,524013,http://tudu-free.blogspot.com/2008/02/jogos-ja...,http://www.phishtank.com/phish_detail.php?phis...,2008-10-10T12:16:44+00:00,yes,2008-10-12T09:08:49+00:00,yes,Other


In [14]:
df_pishing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15261 entries, 0 to 15260
Data columns (total 8 columns):
phish_id             15261 non-null int64
url                  15261 non-null object
phish_detail_url     15261 non-null object
submission_time      15261 non-null object
verified             15261 non-null object
verification_time    15261 non-null object
online               15261 non-null object
target               15261 non-null object
dtypes: int64(1), object(7)
memory usage: 953.9+ KB


In [16]:
df_pishing.shape

(15261, 8)

**benign url은 약 1000000개, phisihng url은 15000개로 데이터 불균형이 있다.**

# Feature Engneering (Phishing URL)
- count number of dots 
- count number of delimeters(';', '_', '?', '=', '&')
- ID addr check
- hyphens('_')
- check '@'
- check `//`
- check `/`
- Return the filename extension from url, or ''
- count number of subdomain
- urlparse

- Digit Count of URL
- URL 길이
- Checking URL is Typosquatted or not (google.com --> goggle.com)
- Subdomain 개수
- Top level Domain 이 자주 쓰이는 Domain인가?
- 등록후 지난 일수
- http:// or https://
- #of http:// or https://
- 특수문자(#, =, /,&)
- com 개수

In [21]:
#2018's top most suspicious TLD 
Suspicious_TLD_18=['country', 'stream', 'download', 'xin', 'gdn', 'racing', 'jetzt', 'win', 'bid', 'vip', 'ren',\
                   'kim', 'loan', 'mom', 'party', 'review', 'trade', 'data', 'wang', 'accountants']

#2019's The 10 Most Abused TLD
Suspicious_TLD_19 = ['gdn', 'ryukyu', 'work', 'fit', 'world', 'okinawa', 'life', 'desi',\
                     'colud', 'ooo']

Suspicious_Domain=['luckytime.co.kr','mattfoll.eu.interia.pl','trafficholder.com','dl.baixaki.com.br','bembed.redtube.comr','tags.expo9.exponential.com','deepspacer.com','funad.co.kr','trafficconverter.biz']

In [22]:
# count number of dots
def countdots(url):
    return url.count('.')

df_pishing['count_dots'] = df_pishing['url'].apply(lambda row: countdots(row))
df_pishing['count_dots_detail_url'] = df_pishing['phish_detail_url'].apply(lambda row: countdots(row))

In [23]:
# count number of delimeters
def countdelim(url):
    count = 0
    delim = [';', '_', '?', '=', '&']
    for each in url:
        if each in delim:
            count = count + 1
    return count

df_pishing['count_delim'] = df_pishing['url'].apply(lambda row: countdelim(row))
df_pishing['count_delim_detail_url'] =df_pishing['phish_detail_url'].apply(lambda row: countdelim(row))

In [24]:
# ID addr check
import ipaddress as ip

def isip(url):
    try:
        if ip.ip_adress(url):
            return 1
    except:
        return 0
df_pishing['ip_check'] = df_pishing['url'].apply(lambda row: isip(row))
df_pishing['ip_check_detail_url'] = df_pishing['phish_detail_url'].apply(lambda row: isip(row))

In [25]:
# hyphens
def isPresentHyphen(url):
    return url.count('-')
df_pishing['count_hyphen'] = df_pishing['url'].apply(lambda row: isPresentHyphen(row))

In [27]:
# check @
def isPresentAt(url):
    return url.count('@')
df_pishing['count_at'] = df_pishing['url'].apply(lambda row: isPresentAt(row))
df_pishing['count_at_detail_url'] = df_pishing['phish_detail_url'].apply(lambda row: isPresentAt(row))

In [28]:
# check `//`
def isPresentDSlash(url):
    return url.count('//')
df_pishing['count_DSlach'] = df_pishing['url'].apply(lambda row: isPresentDSlash(row))
df_pishing['count_DSlach_detail_url'] = df_pishing['phish_detail_url'].apply(lambda row: isPresentDSlash(row))

In [29]:
# check `/`
def countSubDir(url):
    return url.count('/')
df_pishing['count_SubDir'] = df_pishing['url'].apply(lambda row: countSubDir(row))
df_pishing['count_SubDir_detail_url'] = df_pishing['phish_detail_url'].apply(lambda row: countSubDir(row))

In [30]:
# Return the filename extension from url, or ''
import os
def get_ext(url):
    
    root, ext = os.path.splitext(url)
    return ext
df_pishing['ext'] = df_pishing['url'].apply(lambda row: get_ext(row))
df_pishing['ext_detail_url'] = df_pishing['phish_detail_url'].apply(lambda row: get_ext(row))

In [33]:
# count number of subdomain
def countSubDomain(subdomain):
    if not subdomain:
        return 0
    else:
        return len(subdomain.split('.'))
    
df_pishing['count_SubDomain'] = df_pishing['ext'].apply(lambda row: countSubDomain(row)) 

In [36]:
# urlparse
df_pishing['temp'] = df_pishing['url'].apply(lambda url: list(urlparse(url)))
columns = ['scheme', 'netloc', 'path', 'params', 'query', 'fragment']
df_pishing[columns] = pd.DataFrame(df_pishing.temp.values.tolist(), index=df_pishing.index)
df_pishing.drop('temp', axis=1, inplace=True)

In [38]:
df_pishing.tail()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target,count_dots,count_dots_detail_url,...,count_SubDir_detail_url,ext,ext_detail_url,count_SubDomain,scheme,netloc,path,params,query,fragment
15256,1095720,https://sites.google.com/site/freehabbocoinsgb...,http://www.phishtank.com/phish_detail.php?phis...,2010-12-30T12:13:46+00:00,yes,2010-12-30T15:29:19+00:00,yes,Sulake Corporation,2,3,...,3,,.php?phish_id=1095720,0,https,sites.google.com,/site/freehabbocoinsgbbo00/,,,
15257,876840,http://mundovirtualhabbo.blogspot.com/2009_01_...,http://www.phishtank.com/phish_detail.php?phis...,2009-12-03T10:40:26+00:00,yes,2009-12-03T23:56:01+00:00,yes,Sulake Corporation,3,3,...,3,.html,.php?phish_id=876840,2,http,mundovirtualhabbo.blogspot.com,/2009_01_01_archive.html,,,
15258,549159,http://aijcs.blogspot.com/2005/03/colourful-li...,http://www.phishtank.com/phish_detail.php?phis...,2008-11-07T15:04:00+00:00,yes,2011-09-03T19:15:33+00:00,yes,Other,3,3,...,3,.html,.php?phish_id=549159,2,http,aijcs.blogspot.com,/2005/03/colourful-life-of-aij.html,,,
15259,530149,http://tnet.at.ua/index/0-13,http://www.phishtank.com/phish_detail.php?phis...,2008-10-20T09:43:48+00:00,yes,2008-10-21T00:33:18+00:00,yes,Other,2,3,...,3,,.php?phish_id=530149,0,http,tnet.at.ua,/index/0-13,,,
15260,524013,http://tudu-free.blogspot.com/2008/02/jogos-ja...,http://www.phishtank.com/phish_detail.php?phis...,2008-10-10T12:16:44+00:00,yes,2008-10-12T09:08:49+00:00,yes,Other,3,3,...,3,.html#footer-wrap2,.php?phish_id=524013,2,http,tudu-free.blogspot.com,/2008/02/jogos-java-aplicativos.html,,,footer-wrap2


In [40]:
pd.read_csv('../input/df_feature.csv')

,Unnamed: 0,url,label,length,sharp,equal,slash,dbleslash,question,com,...,count_DSlach,count_SubDir,ext,count_SubDomain,scheme,netloc,path,params,query,fragment
0,428982,http://www.meigaoyi.com,0,23,0,0,0,0,0,1,...,1,2,.com,2,http,www.meigaoyi.com,NaN,NaN,NaN,NaN
1,1038946,http://www.cybertruffle.org.uk/vinales/eng/sch...,0,68,0,0,3,0,0,0,...,1,5,.htm,2,http,www.cybertruffle.org.uk,/vinales/eng/schefflera_morototoni.htm,NaN,NaN,NaN
2,20346,http://prague.tv/funny-pictures-archive/,0,40,0,0,2,0,0,0,...,1,4,NaN,0,http,prague.tv,/funny-pictures-archive/,NaN,NaN,NaN
3,869085,http://www.f1autographs.com,0,27,0,0,0,0,0,1,...,1,2,.com,2,http,www.f1autographs.com,NaN,NaN,NaN,NaN
4,925855,http://members3.boardhost.com/ratterriers/,0,42,0,0,2,0,0,1,...,1,4,NaN,0,http,members3.boardhost.com,/ratterriers/,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,14891,http://www.pickleball.global/brentwole/newwire...,1,94,0,0,5,0,0,0,...,1,7,NaN,0,http,www.pickleball.global,/brentwole/newwiresult/garyspeed/6ad6e0f038222...,NaN,NaN,NaN
59996,12426,https://drive.google.com/file/d/1A7vwjAEj1CoCr...,1,82,0,1,4,0,1,1,...,1,6,NaN,0,https,drive.google.com,/file/d/1A7vwjAEj1CoCrJ08YfTU1tU-_crFHPwD/view,NaN,usp=sharing,NaN
59997,2539,http://www.auto-ecole-rennes.com/wp-includes/T...,1,90,0,0,6,0,0,1,...,1,8,NaN,0,http,www.auto-ecole-rennes.com,/wp-includes/Text/Diff/83b3926efe21b0905f3eddd...,NaN,NaN,NaN
59998,5921,http://coolestspotontheweb.com/wp-content/plug...,1,149,0,0,10,0,0,1,...,1,12,NaN,0,http,coolestspotontheweb.com,/wp-content/plugins/ubh/agrement/a78b30fdd6908...,NaN,NaN,NaN


In [41]:
pd.read_csv('../input/df.csv')

,Unnamed: 0,url,label
0,428982,http://www.meigaoyi.com,0
1,1038946,http://www.cybertruffle.org.uk/vinales/eng/sch...,0
2,20346,http://prague.tv/funny-pictures-archive/,0
3,869085,http://www.f1autographs.com,0
4,925855,http://members3.boardhost.com/ratterriers/,0
...,...,...,...
59995,14891,http://www.pickleball.global/brentwole/newwire...,1
59996,12426,https://drive.google.com/file/d/1A7vwjAEj1CoCr...,1
59997,2539,http://www.auto-ecole-rennes.com/wp-includes/T...,1
59998,5921,http://coolestspotontheweb.com/wp-content/plug...,1


In [46]:
import pickle

with open('../input/url_data.pkl', 'rb') as f:
    df = pickle.load(f)
df

,url,label
0,http://www.cscapitaladvisors.com/,0
1,http://www.linuxfestnorthwest.org/,0
2,http://www.funnyfeelingsllc.com/,0
3,http://www.freddyt.com/,0
4,http://www.foodreference.com/html/cheddarbacsa...,0
...,...,...
60256,https://sites.google.com/site/freehabbocoinsgb...,1
60257,http://mundovirtualhabbo.blogspot.com/2009_01_...,1
60258,http://aijcs.blogspot.com/2005/03/colourful-li...,1
60259,http://tnet.at.ua/index/0-13,1
